<!--
#  Copyright Amazon.com, Inc. or its affiliates. All Rights Reserved.
#
#    Licensed under the Apache License, Version 2.0 (the "License").
#    You may not use this file except in compliance with the License.
#    You may obtain a copy of the License at
#
#        http://www.apache.org/licenses/LICENSE-2.0
#
#    Unless required by applicable law or agreed to in writing, software
#    distributed under the License is distributed on an "AS IS" BASIS,
#    WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
#    See the License for the specific language governing permissions and
#    limitations under the License.
-->

# Sample notebook to build custom images from the lake user teamspace.

## Content
1. [Configuration](#Configuration)
2. [Build Image](#Build-Image)
2. [Running Container using the profile](#Running-container)


### Configuration

In [ ]:
profile_name='simple_image'
image_name = 'custom_image2'
folder_name = 'simple'

### Build Image

Lets see how  orbit build image works...

In [ ]:
!orbit build image --help

get our orbit env and team names

In [ ]:
env_name = %env AWS_ORBIT_ENV
team_name = %env AWS_ORBIT_TEAM_SPACE
user_name = %env USERNAME
namespace = %env AWS_ORBIT_USER_SPACE
(env_name,team_name, user_name, namespace)

Repository name will be created from the image name prefixed by the env context.  Users are only able to manipulate ECR repos that start with 'orbit-{env_name}/users/'

In [ ]:
repository_name = (f"orbit-{env_name}/users/{image_name}")
repository_name

In [ ]:
!aws ecr delete-repository --repository-name $repository_name --force

In [ ]:
%cd ~/shared/samples/notebooks/I-Image/$folder_name

adding a file to our docker as an example

In [ ]:
%%writefile my-custom-image-file.txt
just a text 

In [ ]:
pwd = %pwd
pwd

Now lets run the command

In [ ]:
%%time

output = !orbit build image -e $env_name -d $pwd -n $image_name
output

Lets get the image address from the output of the previous command

In [ ]:
look_for = 'ECR Image Address='
image = None
for o in output:
    if look_for in o:
        image = o[o.index(look_for) + len(look_for):]
        print(image)

assert(image != None)       
    

In [ ]:
# check that the image was built
import json
print(repository_name)
images = !aws ecr list-images --repository-name $repository_name
images = "".join(images)
im = json.loads(images)
print(im['imageIds'])
assert(len(im['imageIds']) > 0)

### Building the PodSetting for the Image

In [ ]:
import json

customname = "orbit-custom-simple-image-"+team_name
description= "Custom Image Example "+customname
podsetting={
            "name": customname,
            "description": description,
            "image": image,
            "resources":{
                "limits":{            
                    "cpu": "1.0",
                    "memory": "1Gi"
                    },
                "requests":{
                    "cpu": "1.0",
                    "memory": "1Gi"
                    }
            },
            "env":[
                {
                    "name":"source",
                    "value":"regressiontests"
                }
            ]
        }

with open("podsetting_data.json", 'w') as f:
    json.dump(podsetting, f)

In [ ]:
!orbit build podsetting --help 

In [ ]:
!orbit build podsetting --debug -e $env_name -t $team_name  podsetting_data.json

In [ ]:
!pwd

In [ ]:
import json
run = {
      "compute": {
          "container" : {
              "p_concurrent": "1"
          },
          "node_type": "ec2",
          "podsetting":customname
      },
      "tasks":  [{
          "notebookName": "test-image.ipynb",
          "sourcePath": pwd,
          "targetPath": f"/home/jovyan/shared/regression/notebooks/I-Image/{folder_name}",
          "params": {
          }
        }]
 }

with open("run.json", 'w') as f:
    json.dump(run, f)

### Running container 

Lets run a container using the profile and image we created

In [ ]:
%%time

!orbit run notebook --env $env_name --team $team_name --user testing --wait --tail-logs run.json


## OPTIONAL: You can use kubectl to load the podsettings as indicated below 

### (convert the cells to code from markdown)

import json

customname = "orbit-custom-simple-image-"+team_name

with open("podsetting_simple.yaml", "w") as file:
    file.write("""
    kind: PodSetting
    apiVersion: orbit.aws/v1
    metadata:
      labels:
        orbit/env: {env_name}
        orbit/space: team
        orbit/team: {team_name}
      name: {customname}
      namespace: {team_name}
    spec:
      containerSelector:
        jsonpath: metadata.labels.app
      desc: Custom Image Example {customname}
      env:
        - name: custom_name
          value: custom_value
      image: >-
        {image}
      podSelector:
        matchExpressions:
          - key: orbit/{customname}
            operator: Exists
      resources:
        limits:
          cpu: '1.0'
          memory: 1Gi
        requests:
          cpu: '1.0'
          memory: 1Gi
      securityContext:
        runAsUser: 1000
    """.format(team_name=team_name,env_name=env_name,customname=customname,image=image)
)


!kubectl apply -f podsetting_simple.yaml -n {team_name}